In [1]:
import pandas as pd
import requests
import datetime
api_key = '25MRYYUQTU9JVAURPS88G9E2JFFKJIBFZJ'

url = "https://api.etherscan.io/api"
from TDD import TDD

### **Functions**

In [2]:
def first_last_info(address):

    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "page": 1,
        "offset": 100,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": api_key
    }

    
    response = requests.get(url, params=params).json()
    response = response['result']
    first_date = response[0]['timeStamp']
    last_date = response[-1]['timeStamp']

    first_to = response[0]['to']
    first_from = response[0]['from']

    last_to = response[-1]['to']
    last_from = response[-1]['from']

    if first_to == address:
        first_to = 'self'
    if first_from == address:
        first_from = 'self'
    if last_to == address:
        last_to = 'self'
    if last_from == address:
        last_from = 'self'
 
    return [address,first_date,last_date,first_from,first_to,last_from,last_to]

In [3]:
def get_transaction_history(address):

    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "page": 1,
        "offset": 100,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": api_key
    }

    
    response = requests.get(url, params=params).json()
    return response['result']

def get_Erc20_transaction_history(address):

    params = {
        "module": "account",
        "action": "tokentx",
        "address": address,
        "page": 1,
        "offset": 100,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": api_key
    }

    
    response = requests.get(url, params=params).json()
    return response['result']

def get_wallet_age(history: list[dict]):
    if len(history) > 0:
        creation_time = int(history[0]['timeStamp'])
        creation_date = datetime.datetime.fromtimestamp(creation_time).date()
        current_date = datetime.date.today()
        wallet_age = (current_date - creation_date).days
        return wallet_age
    else:
        return 0

def to_and_from(history:list[dict],address):
    from_count = 0
    to_count = 0
    for transactions in history:
        if transactions['from'] == address:
            from_count +=1
        else:
            to_count += 1
    return from_count, to_count

In [4]:
def fetch(address,nested_list):

    reg_hist = get_transaction_history(address)
    erc20_hist = get_Erc20_transaction_history(address)

    reg_age = get_wallet_age(reg_hist)
    erc_age = get_wallet_age(erc20_hist)

    reg_to,reg_from = to_and_from(reg_hist,address)
    erc_to,erc_from = to_and_from(erc20_hist,address)

    tdd,txn_count = TDD(address)

    row = [address,txn_count,reg_age,erc_age,reg_to,reg_from,erc_to,erc_from,tdd]
    nested_list.append(row)

## **Alpha ETH**

In [5]:
Alpha_Eth = pd.read_csv('voters/alpha_eth.csv')
Infra_sybils = pd.read_excel('research_data/alpha_round_sybils.xlsx', sheet_name='Eth Infra')
Alpha_Eth.dropna(subset=['supporterwallet'],inplace=True)

In [6]:
known_sybils = Infra_sybils.loc[(Infra_sybils['Suspected sybil?']==True)&(Infra_sybils['Reviewed Manually']==True)].copy()
known_sybils.rename(columns={'Source Address': 'supporterwallet'},inplace=True)

merged = pd.merge(left=known_sybils,right = Alpha_Eth,how='left',on='supporterwallet')
merged.drop(['ID', 'Token', 'Amount','Etherscan','projectid','txnhash','communityid'], axis=1, inplace=True)

In [7]:
address_counts = merged['supporterwallet'].value_counts()
Address_info_sybil = pd.DataFrame({'Address': address_counts.index, 'Count': address_counts.values})

In [8]:
funding_counts = Alpha_Eth['supporterwallet'].value_counts()
count_by_address_project = Alpha_Eth.groupby(['supporterwallet', 'project_title']).size().reset_index(name='count')
no_grants_funded = count_by_address_project['supporterwallet'].value_counts()

# create a new dataframe with the address counts as a column
Address_info1 = pd.DataFrame({'supporterwallet': funding_counts.index, 'Funding_count': funding_counts.values})
Address_info2 = pd.DataFrame({'supporterwallet': no_grants_funded.index, 'No_Grants_Funded': no_grants_funded.values})
Address_info_eth = pd.merge(Address_info1,Address_info2)

Address_info_eth.shape[0] == len(Alpha_Eth['supporterwallet'].unique())

True

In [9]:
Address_info_eth.describe()

,Funding_count,No_Grants_Funded
count,4777.000000,4777.000000
mean,4.382248,4.244714
std,5.236653,5.001276
min,1.000000,1.000000
25%,1.000000,1.000000
50%,2.000000,2.000000
75%,5.000000,5.000000
max,44.000000,22.000000


In [10]:
Address_info_eth_filtered = Address_info_eth.loc[(Address_info_eth['No_Grants_Funded']<=5)&(Address_info_eth['Funding_count']<=5)].reset_index(drop=True)

In [11]:
filtered = pd.merge(Address_info_eth_filtered,Alpha_Eth,how='left',on='supporterwallet')
filtered = filtered.drop(['projectid','txnhash','exchangerate','updatedat'],axis=1)
data_points = ['supporterwallet','Funding_count', 'No_Grants_Funded','token', 'amount', 'createdat','passportscore','passportpass']

### Querying_eth_infra

In [12]:
# headers = ['supporterwallet','txn_count','Wallet_Age','Wallet_Age(Erc20)','to_count','from_count','erc_to','erc_from','TDD']
# contents = []

# count = 0
# for i in merged['supporterwallet'].unique():
#     print(count)
#     fetch(i,contents)
#     count+=1

#sybil_alpha_sybil_data = pd.DataFrame(contents,columns=headers)
#sybil_alpha_sybil_data.to_csv('queried_data/eth_infra_sybil_data.csv',index=False)

In [13]:
# headers = ['supporterwallet','first_date','last_date','first_from','first_to','last_from','last_to']
# contents = []

# count = 0
# for i in eth_infra_q['supporterwallet']:
#     print(count)
#     info = first_last_info(i)
#     contents.append(info)
#     count+=1

# data = pd.DataFrame(contents,columns=headers)

In [14]:
# count = 0
# for i in filtered['supporterwallet'].unique():
#     print(count)
#     fetch(i,contents)
#     count+=1

# alpha_sybil_data = pd.DataFrame(contents,columns=headers)

# eth_infra_q = pd.merge(data,alpha_sybil_data,on='supporterwallet')

# alpha_sybil_data.to_csv('queried_data/eth_infra_data.csv',index=False)

In [15]:
eth_infra_q = pd.read_csv('queried_data/eth_infra_data.csv')

In [16]:
filtered['address'] = filtered['supporterwallet']

filtered['project_title_sorted'] = filtered['project_title'].apply(lambda x: '-'.join(sorted(x.lower().split())))

# group the rows by the address value and apply aggregation functions to the columns
df_result = filtered.groupby('address').agg({'supporterwallet': 'first',
                                       'project_title_sorted': '_'.join}).reset_index()

# sort the resulting DataFrame by the count of project titles in descending order
df_result = df_result.sort_values(by='project_title_sorted', ascending=False)[['supporterwallet','project_title_sorted']]

cut_filtered = filtered[data_points].drop_duplicates(subset=['supporterwallet'])

cultivate_data_infra = pd.merge(cut_filtered,df_result,on='supporterwallet',how='left')

## **Beta Round**

In [90]:
beta = pd.read_csv('voters/beta_round_votes.csv')
beta.dropna(subset=['voter'],inplace=True)

In [91]:
beta.rename(columns={'voter':'supporterwallet','grantAddress':'project_addresses'},inplace=True)

In [92]:
funding_counts = beta['supporterwallet'].value_counts()
count_by_address_project = beta.groupby(['supporterwallet', 'project_addresses']).size().reset_index(name='count')
no_grants_funded = count_by_address_project['supporterwallet'].value_counts()

# create a new dataframe with the address counts as a column
Address_info_beta_1 = pd.DataFrame({'supporterwallet': funding_counts.index, 'Funding_count': funding_counts.values})
Address_info_beta_2 = pd.DataFrame({'supporterwallet': no_grants_funded.index, 'No_Grants_Funded': no_grants_funded.values})
Address_info_beta = pd.merge(Address_info_beta_1,Address_info_beta_2)

Address_info_beta.shape[0] == len(beta['supporterwallet'].unique())

True

In [93]:
Address_info_beta_filtered = Address_info_beta.loc[(Address_info_beta['No_Grants_Funded']<=5)&(Address_info_beta['Funding_count']<=5)].reset_index(drop=True)

In [105]:
filtered_beta = pd.merge(Address_info_beta_filtered,beta,how='left',on='supporterwallet')
filtered_beta = filtered_beta.drop(['id','transaction', 'blockNumber','projectId', 'applicationId', 'roundId', 'amount', 'amountUSD','amountRoundToken','token'],axis=1)

In [106]:
len(filtered_beta['supporterwallet'].unique())

4267

### Querying beta

In [107]:
# headers = ['supporterwallet','first_date','last_date','first_from','first_to','last_from','last_to']
# contents = []

# count = 4051
# for i in filtered_beta['supporterwallet'].unique()[4051:]:
#     print(count)
#     info = first_last_info(i)
#     contents.append(info)
#     count+=1

# data = pd.DataFrame(contents,columns=headers)

In [108]:
# headers = ['supporterwallet','txn_count','Wallet_Age','Wallet_Age(Erc20)','to_count','from_count','erc_to','erc_from','TDD']
# contents = []

# count = 0
# for i in filtered_beta['supporterwallet'].unique():
#     print(count)
#     fetch(i,contents)
#     count+=1


# beta_data = pd.DataFrame(contents,columns=headers)
# df = pd.merge(beta_data,data,on='supporterwallet')
# df.to_csv('queried_data/beta_data.csv',index=False)

In [134]:
beta_data = pd.read_csv('queried_data/beta_data.csv')

In [135]:
beta_points =['supporterwallet','Funding_count','No_Grants_Funded']

In [136]:
filtered_beta['address'] = filtered_beta['supporterwallet']

filtered_beta['project_title_sorted'] = filtered_beta['project_addresses'].apply(lambda x: '-'.join(sorted(x.lower().split())))

# group the rows by the address value and apply aggregation functions to the columns
df_result = filtered_beta.groupby('address').agg({'supporterwallet': 'first',
                                       'project_title_sorted': '_'.join}).reset_index()

# sort the resulting DataFrame by the count of project titles in descending order
df_result = df_result.sort_values(by='project_title_sorted', ascending=False)[['supporterwallet','project_title_sorted']]

cut_filtered_beta = filtered_beta[beta_points].drop_duplicates(subset=['supporterwallet'])

cultivate_data_beta = pd.merge(cut_filtered_beta,df_result,on='supporterwallet',how='left')

## **Grouping**

In [206]:
stats_data_infra = pd.merge(cultivate_data_infra,eth_infra_q,on='supporterwallet')
stats_data_beta = pd.merge(cultivate_data_beta,beta_data,on='supporterwallet')
stats_data_beta.drop_duplicates(subset='supporterwallet',inplace=True)

In [207]:
stats_data_infra = stats_data_infra.reindex(sorted(stats_data_infra.columns), axis=1)
stats_data_beta = stats_data_beta.reindex(sorted(stats_data_beta.columns), axis=1)

In [208]:
stats_data_beta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4267 entries, 0 to 4274
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Funding_count         4267 non-null   int64  
 1   No_Grants_Funded      4267 non-null   int64  
 2   TDD                   4267 non-null   float64
 3   Wallet_Age            4267 non-null   int64  
 4   Wallet_Age(Erc20)     4267 non-null   int64  
 5   erc_from              4267 non-null   int64  
 6   erc_to                4267 non-null   int64  
 7   first_date            4267 non-null   int64  
 8   first_from            4267 non-null   object 
 9   first_to              4267 non-null   object 
 10  from_count            4267 non-null   int64  
 11  last_date             4267 non-null   int64  
 12  last_from             4267 non-null   object 
 13  last_to               4267 non-null   object 
 14  project_title_sorted  4267 non-null   object 
 15  supporterwallet      

In [209]:
stats_data_beta.fillna(0.0,inplace=True)
stats_data_infra.fillna(0.0,inplace=True)

In [210]:
stats_data_i = stats_data_infra.copy()
stats_data_b = stats_data_beta.copy()


In [211]:
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ['token', 'project_title_sorted', 'first_from', 'first_to', 'last_from', 'last_to']

for col in columns_to_encode:
    le = LabelEncoder()
    stats_data_i[col] = le.fit_transform(stats_data_i[col])

columns_to_encode = ['project_title_sorted', 'first_from', 'first_to', 'last_from', 'last_to']

for col in columns_to_encode:
    le = LabelEncoder()
    stats_data_b[col] = le.fit_transform(stats_data_b[col])

In [212]:
feature_infra = ['Funding_count', 'No_Grants_Funded', 'token','amount', 'passportscore', 'passportpass',
            'project_title_sorted','first_from','first_to', 'last_from', 'last_to', 'txn_count', 'Wallet_Age',
            'Wallet_Age(Erc20)', 'to_count', 'from_count', 'erc_to', 'erc_from']

###  Eth Infra

In [213]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

data = np.array(stats_data_i[feature_infra])

# Add supporterwallet column to data
supporterwallet_col = stats_data_i['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.9999

# Initialize list to store similar supporter wallets
similar_supporters_infra = []

# Loop through similarity matrix and group together similar supporter wallets
for i in range(len(similarity_matrix)):
    similar_supporter_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_supporter_indices) > 1:
        similar_supporter_wallets = [data[j][0] for j in similar_supporter_indices]
        for supporter in similar_supporter_wallets:
            if supporter not in similar_supporters_infra:
                similar_supporters_infra.append(supporter)

In [214]:
data = stats_data_i[feature_infra].astype(float)

# Add supporterwallet column to data
supporterwallet_col = stats_data_i['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.9999

# Initialize list to store similar rows
similar_rows = []

# Loop through similarity matrix and group together similar rows
for i in range(len(similarity_matrix)):
    similar_row_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_row_indices) > 1:
        similar_row_values = [tuple(data[j]) for j in similar_row_indices]
        if similar_row_values not in similar_rows:
            similar_rows.append(similar_row_values)

# Print out the similar rows
for i, row_group in enumerate(similar_rows):
    print(f"Similar Row Group {i}:")
    for row in row_group:
        print(row[0], row[1:])
    print()


Similar Row Group 0:
0x992129ad0e5c642f4d088ba6b51da823588b09e1 (4.0, 4.0, 1.0, 0.0019, 22.24, 1.0, 310.0, 1221.0, 90.0, 151.0, 493.0, 29.0, 160.0, 158.0, 29.0, 8.0, 1.0, 2.0)
0x8be31a599d12b6bf92a21adc211b0065505915af (4.0, 4.0, 1.0, 0.0019407, 22.22, 1.0, 310.0, 1221.0, 90.0, 151.0, 493.0, 33.0, 160.0, 158.0, 33.0, 9.0, 1.0, 2.0)

Similar Row Group 1:
0xfbbd5070addc38f53f95bc5259c2d0f9e90a92eb (3.0, 3.0, 1.0, 0.00065, 35.69, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 46.0, 675.0, 704.0, 46.0, 26.0, 3.0, 5.0)
0xfa6b1a941c8f9515ab2122b47b4c806efb13b7b9 (3.0, 3.0, 1.0, 0.00065, 35.69, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 50.0, 675.0, 704.0, 50.0, 26.0, 3.0, 6.0)
0xa879b4fef26dfd1cd35843ddd6071f2bc63ebfb7 (3.0, 3.0, 1.0, 0.00065, 35.69, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 45.0, 675.0, 704.0, 45.0, 25.0, 3.0, 6.0)
0xb03ae245b2bb3526fdb511954b3775f11db12458 (3.0, 3.0, 1.0, 0.00065, 30.71, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 44.0, 675.0, 704.0, 44.0, 26.0, 3.0, 7.0)

Similar Row

In [215]:
similar_supporters_infra

['0x992129ad0e5c642f4d088ba6b51da823588b09e1',
 '0x8be31a599d12b6bf92a21adc211b0065505915af',
 '0xfbbd5070addc38f53f95bc5259c2d0f9e90a92eb',
 '0xfa6b1a941c8f9515ab2122b47b4c806efb13b7b9',
 '0xa879b4fef26dfd1cd35843ddd6071f2bc63ebfb7',
 '0xb03ae245b2bb3526fdb511954b3775f11db12458',
 '0xfdee0c33f3493a1b83cc63bb5fbfa766572fbc03',
 '0xc900edd003feea41c0efb57262ba3d70608e7e22',
 '0xfc888fadd4063cb98041a59a232183a6d4306da2',
 '0xe6a1ff1cdb17130212e8a1ff596b8beadf112735',
 '0xcbc0012dd50f9e3920d55ea25aa6a22104c1c9ca',
 '0x90b5c0a0f414c7824d54802c5ab0c9fb10fb82d8',
 '0x508d4589ba566f94edb89a4ab15466ee128dffc3',
 '0x7e646d9f5ee2947c66e87aa414f8a5e7b26dd380',
 '0x5f0eb11a1232a771fcb91f7f796a5ee4cb217217',
 '0x68c882487920d8c114b9f80c4bbf882f60befc5b',
 '0xa2b1c8798cac94cab5a7b493a473979a724e1a1c',
 '0x2e884770f9e11ba6b649039e34f7504ada6ad8bb',
 '0x29d6f812e973dcbbc2e28d3c24c4baf371afd893',
 '0xc6d0b227ffcb0c7bb7eef6cde19e173a6b3ea7d5',
 '0xbdbe9e680d06fa764976d7ef2eee8cece745051d',
 '0x4e088ee46

In [216]:
infra_identified_sybils = pd.DataFrame(similar_supporters_infra,columns=['address'])
infra_identified_sybils.to_csv('identified_sybils/eth_infra.csv',index=False)

### Beta

In [233]:
feature_beta = ['Funding_count', 'No_Grants_Funded', 'TDD', 'Wallet_Age',
       'Wallet_Age(Erc20)', 'erc_from', 'erc_to',
       'first_to', 'from_count',
       'project_title_sorted', 'to_count', 'txn_count']

In [234]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

data = np.array(stats_data_b[feature_beta])

# Add supporterwallet column to data
supporterwallet_col = stats_data_b['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.999999

# Initialize list to store similar supporter wallets
similar_supporters_beta = []

# Loop through similarity matrix and group together similar supporter wallets
for i in range(len(similarity_matrix)):
    similar_supporter_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_supporter_indices) > 1:
        similar_supporter_wallets = [data[j][0] for j in similar_supporter_indices]
        for supporter in similar_supporter_wallets:
            if supporter not in similar_supporters_infra:
                similar_supporters_beta.append(supporter)

In [235]:
similar_supporters_beta

['0xEeD135f9F1182c8d5b024E321663937DbfADeDb4',
 '0xeE9fc27f84f7B67db334e829B297974F36E32e85',
 '0x8A5C62990eeC0e3D794B5444886aB3c105Fb03d4',
 '0x8bA2a37eD2a6606F5897A3145435bf830E434f3b',
 '0x633D4Cd6bB4d0c227792537Ad006705A4191cE02',
 '0xb62462465E2697F2275F668E775710Fa24fC9394',
 '0xfed55eEE395082bbF2595bacAb8798BE98CBD232',
 '0xffbeE70C7E4360c6dF4c7Aa834566Ba4eFEF6669',
 '0xfed55eEE395082bbF2595bacAb8798BE98CBD232',
 '0xffbeE70C7E4360c6dF4c7Aa834566Ba4eFEF6669',
 '0xEeD135f9F1182c8d5b024E321663937DbfADeDb4',
 '0xeE9fc27f84f7B67db334e829B297974F36E32e85',
 '0xc098b08b4D7dC5deea18Bc917ab2d5aE542910D0',
 '0xBFbd5547568D9F31Fd4E6A1Dd0fce342255D4210',
 '0xc098b08b4D7dC5deea18Bc917ab2d5aE542910D0',
 '0xBFbd5547568D9F31Fd4E6A1Dd0fce342255D4210',
 '0xD8Fd11d65f3f9805e40526892f3034C75a2580D3',
 '0xd8c2B741cb31e5352dC32fb3F0819D03A2c4c01c',
 '0xD8Fd11d65f3f9805e40526892f3034C75a2580D3',
 '0xd8c2B741cb31e5352dC32fb3F0819D03A2c4c01c',
 '0x9234938dB2d9bD6D944d27f4F7b17187c81a5d7A',
 '0x7BE839eA6

In [237]:
data = stats_data_b[feature_beta]

# Add supporterwallet column to data
supporterwallet_col = stats_data_b['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.9999992

# Initialize list to store similar rows
similar_rows = []

# Loop through similarity matrix and group together similar rows
for i in range(len(similarity_matrix)):
    similar_row_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_row_indices) > 1:
        similar_row_values = [tuple(data[j]) for j in similar_row_indices]
        if similar_row_values not in similar_rows:
            similar_rows.append(similar_row_values)

# Print out the similar rows
for i, row_group in enumerate(similar_rows):
    print(f"Similar Row Group {i}:")
    for row in row_group:
        print(row[0], row[1:])
    print()


Similar Row Group 0:
0xEeD135f9F1182c8d5b024E321663937DbfADeDb4 (4.0, 4.0, 15.63157894736842, 313.0, 233.0, 1.0, 0.0, 3796.0, 26.0, 381.0, 0.0, 19.0)
0xeE9fc27f84f7B67db334e829B297974F36E32e85 (4.0, 4.0, 14.285714285714286, 314.0, 233.0, 1.0, 0.0, 3792.0, 28.0, 381.0, 0.0, 21.0)

Similar Row Group 1:
0x8A5C62990eeC0e3D794B5444886aB3c105Fb03d4 (4.0, 4.0, 22.846153846153847, 313.0, 0.0, 0.0, 0.0, 2210.0, 19.0, 166.0, 0.0, 13.0)
0x8bA2a37eD2a6606F5897A3145435bf830E434f3b (4.0, 4.0, 23.07692307692308, 313.0, 0.0, 0.0, 0.0, 2226.0, 18.0, 166.0, 0.0, 13.0)

Similar Row Group 2:
0x633D4Cd6bB4d0c227792537Ad006705A4191cE02 (4.0, 4.0, 6.619047619047619, 150.0, 71.0, 5.0, 0.0, 154.0, 25.0, 601.0, 0.0, 21.0)
0xb62462465E2697F2275F668E775710Fa24fC9394 (4.0, 4.0, 6.619047619047619, 150.0, 71.0, 5.0, 0.0, 154.0, 25.0, 601.0, 0.0, 21.0)

Similar Row Group 3:
0xc098b08b4D7dC5deea18Bc917ab2d5aE542910D0 (4.0, 4.0, 21.428571428571427, 313.0, 0.0, 0.0, 0.0, 3068.0, 22.0, 166.0, 0.0, 14.0)
0xBFbd5547568D9F3

In [238]:
beta_identified_sybils = pd.DataFrame(similar_supporters_beta,columns=['address'])
beta_identified_sybils.to_csv('identified_sybils/beta.csv',index=False)